In [5]:
import netCDF4
import xarray as xr
import numpy as np
import cartopy.crs as ccrs


# Getting wind speed over land using CDO

In [15]:
ws = xr.open_dataset('./data/wind10_eur_19790101_20191231_LAND.nc')